In [0]:
import os.path as path
import os
import json
from random import randint
import time
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import polygon
import skimage.io as sio

from pycocotools.coco import COCO
from operator import add 
import glob
import math
import random

In [0]:
posetrack_home_fp = path.expanduser('/content/drive/My Drive/Posetrack 2018')
posetrack_annotations_fp = os.path.join(posetrack_home_fp, 'annotations')

assert(os.path.exists(posetrack_home_fp))
assert(os.path.exists(posetrack_annotations_fp))

In [0]:
train_videos_list = []
videos_fp = os.path.join(posetrack_home_fp, 'images')
train_videos_fp = os.path.join(videos_fp, 'train')
print(train_videos_fp)

for filename in glob.iglob(train_videos_fp+"/**", recursive = False):
  train_videos_list.append(filename) 
train_videos_list.sort()                   
print(train_videos_list)
print(len(train_videos_list))

In [0]:
training_video_frames = []
for i, video in enumerate(train_videos_list):
  
  current_video = []
  for filepath in glob.iglob(video+"/**", recursive = False):
    current_video.append(filepath)
  current_video.sort()
  print(str(i+1)+ " video completed")
  training_video_frames.append(current_video)

In [0]:
import os
import numpy as np
train_annotations_fp = os.path.join(posetrack_annotations_fp, 'train')
train_annotations_list = []
for filename in glob.iglob(train_annotations_fp+"/**", recursive = False):
  train_annotations_list.append(filename) 
train_annotations_list.sort()                   
print(train_annotations_list)
print(len(train_annotations_list))

In [0]:
training_anns_data = []
training_anns_id_data = []
training_labels_data = []
training_keypoints = []
training_bbox = []
training_video_labeled_frames = []
for i, annot in enumerate(train_annotations_list):
  current_video_anns = []
  current_video_ann_ids = []
  current_video_keypoints = []
  
  current_video_label = []
  current_video_bbox = []
  current_video_labeled_frame = []
  coco = COCO(annot)
  img_ids = coco.getImgIds()
  imgs = coco.loadImgs(img_ids)


  #print(len(imgs))
  #print((img_ids))
  #print((imgs))
  for image_idx, img in enumerate(imgs):

    current_img_label = img['is_labeled']
    current_video_label.append(current_img_label)
    if(current_img_label == True):
      labeled_frame = os.path.join(posetrack_home_fp, img['file_name'])
      current_video_labeled_frame.append(labeled_frame)
    #if(current_img_label == False):
     # unlabeled_frame = os.path.join(posetrack_home_fp, img['filename'])
    current_img_ann_ids = coco.getAnnIds(imgIds=img['id'])
    current_img_anns = coco.loadAnns(current_img_ann_ids)
    
    current_img_keypoints = []
    cu_img_keypoints = [ann['keypoints'] for ann in current_img_anns]
    for person in cu_img_keypoints:
      current_person_keypoints = []
      for j in range(len(person)):
        if j%3 == 0:
          current_person_keypoints.append((person[j], person[j+1]))
      current_img_keypoints.append(current_person_keypoints)

    
    current_img_bbox = [ann.get('bbox') for ann in current_img_anns]
    
    current_video_anns.append(current_img_anns)
    current_video_ann_ids.append(current_img_ann_ids)
    if len(cu_img_keypoints) != 0:
      current_video_keypoints.append(current_img_keypoints)
    current_video_bbox.append(current_img_bbox)

  training_video_labeled_frames.append(current_video_labeled_frame)
  training_anns_data.append(current_video_anns)
  training_anns_id_data.append(current_video_ann_ids)
  training_labels_data.append(current_video_label)

  training_keypoints.append(current_video_keypoints)
  training_bbox.append(current_video_bbox)
  print(str(i+1)+"video completed")

In [0]:
training_length_bbox = []
for i in range(len(train_videos_list)):
  current = []
  for j in range(len(training_bbox[i])):
    if(len(training_bbox[i][j]) == 0):
      continue
    else:
      current.append(len(training_bbox[i][j]))
  length = min(current)
  training_length_bbox.append(length)

In [0]:
training_video_unlabeled_frames = []
for i, video in enumerate(training_video_frames):
  current_video_unlabeled_frame = []
  frames = training_video_labeled_frames[i]
  for j,frame in enumerate(video):
    if frame in frames:
      continue
    else:
      current_video_unlabeled_frame.append(frame)
  training_video_unlabeled_frames.append(current_video_unlabeled_frame)
 
